# `spacy exploration` notebook
### Descriprion
Notebook investigates `spaCy` facilities to text parsing and NER tasks. There are several examples of `spaCy` behaviour and ways of interacting with this awesome library.

In [1]:
%cd ../..

/home/pristalovya/Документы/nlp-coursework


In [2]:
from datasets import DatasetLoader

import spacy
from spacy import displacy

import pandas as pd

from tqdm import tqdm
from collections import defaultdict
from typing import List
import time
from IPython.display import clear_output
import sys
import dill

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=6, progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
data = DatasetLoader.load_reviews(show_path=True).get(['review', 'film_id'])
data

/home/pristalovya/Документы/nlp-coursework/data/reviews.csv


,review,film_id
0,«Зеленую милю» я смотрела два раза: 10 лет наз...,435
1,Период конца девяностых годов-начало двухтысяч...,435
2,"Очень сложно писать рецензию на этот фильм, та...",435
3,Любимая многими миллионами ценителями киноиску...,435
4,В нашем мире существует много разных фильмов. ...,435
...,...,...
90641,"Конечно, этот фильм - не лучший представитель ...",969760
90642,Фильм «Ламборгини: Человек-легенда» снят в 202...,969760
90643,"Эй, рагацци, вы это серьёзно, ТАК показывать и...",969760
90644,"Вообще, говоря о байопиках, стоит отметить, чт...",969760


In [4]:
# Загружаем модель
nlp = spacy.load('ru_core_news_lg')

sentence = "«Зеленую милю» я смотрела два раза: 10 лет назад и сейчас."

# Передаем текст в модель
doc = nlp(sentence)

In [5]:
doc[:10]

«Зеленую милю» я смотрела два раза: 10

In [44]:
spacy.explain('nsubj')

'nominal subject'

In [6]:
spacy.explain('PRON')

'pronoun'

In [14]:
doc = nlp(data.review[0])

In [15]:
# Outside of jupyter, use displacy.serve()
displacy.render(doc, style='dep', jupyter=True)

In [16]:
displacy.render(doc, style='ent', jupyter=True)

In [17]:
print(doc[1])
print(doc[1].text)

Зеленую
Зеленую


In [18]:
for token in doc[:30]:
    print(f"{token.text:20} {token.pos_:7} {token.dep_:10}")

«                    PUNCT   punct     
Зеленую              ADJ     amod      
милю                 NOUN    obj       
»                    PUNCT   punct     
я                    PRON    nsubj     
смотрела             VERB    ROOT      
два                  NUM     nummod:gov
раза                 NOUN    obl       
:                    PUNCT   punct     
10                   NUM     nummod    
лет                  NOUN    parataxis 
назад                ADV     advmod    
и                    CCONJ   cc        
сейчас               ADV     conj      
.                    PUNCT   punct     


                 SPACE   dep       
Первый               ADJ     amod      
раз                  NOUN    obl       
у                    ADP     case      
меня                 PRON    obl       
было                 VERB    ROOT      
примерно             ADV     advmod    
такое                DET     det       
впечатление          NOUN    nsubj     
,                    PUNCT   punct     
о 

In [63]:
spacy.explain('obl')

'oblique nominal'

In [74]:
from pprint import pprint
dep_term_descriptions = {}
for token in doc:
    dep_term_descriptions[token.dep_] = spacy.explain(token.dep_)
pprint(dep_term_descriptions)

{'ROOT': 'root',
 'acl': 'clausal modifier of noun (adjectival clause)',
 'acl:relcl': None,
 'advcl': 'adverbial clause modifier',
 'advmod': 'adverbial modifier',
 'amod': 'adjectival modifier',
 'case': 'case marking',
 'cc': 'coordinating conjunction',
 'ccomp': 'clausal complement',
 'conj': 'conjunct',
 'cop': 'copula',
 'csubj': 'clausal subject',
 'dep': 'unclassified dependent',
 'det': 'determiner',
 'fixed': 'fixed multiword expression',
 'flat:name': None,
 'iobj': 'indirect object',
 'mark': 'marker',
 'nmod': 'modifier of nominal',
 'nsubj': 'nominal subject',
 'nsubj:pass': None,
 'nummod': 'numeric modifier',
 'nummod:gov': None,
 'obj': 'object',
 'obl': 'oblique nominal',
 'parataxis': 'parataxis',
 'punct': 'punctuation',
 'xcomp': 'open clausal complement'}


In [19]:
print(f'{"entity":<50} {"label":<10} description')
for ent in doc.ents:
    print(f'{ent.text:<50} {ent.label_:<10} {spacy.explain(ent.label_)}')

entity                                             label      description
Кинга                                              PER        Named person or family.
Дарабонта                                          PER        Named person or family.
Перси                                              PER        Named person or family.
Депрессии                                          LOC        Non-GPE locations, mountain ranges, bodies of water
Делакруа                                           PER        Named person or family.
Дарабонт                                           PER        Named person or family.
Майкл Кларк                                        PER        Named person or family.
Дункан                                             PER        Named person or family.
